# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter
import timm

#from torchvision.models import wide_resnet50_2, resnet18
#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
import common as com
import pytorch_modeler as modeler
from pytorch_model import EfficientNet_b1 as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

In [4]:
# effnet1 = timm.create_model('efficientnet_b1', pretrained=True)
# # M7:block[5], M8:block[6], M9:conv_head
# effnet1

## load config and set logger

In [5]:
log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

## Setting

In [6]:
# Setting seed
modeler.set_seed(42)

In [7]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_dir = INPUT_ROOT + "/dev_data"
add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = os.listdir(dev_dir)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'

TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
PRED_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/pred'
SCORE_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/score'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
os.makedirs(PRED_DIR, exist_ok=True)
os.makedirs(SCORE_DIR, exist_ok=True)

# copy config
# shutil.copy('./config.yaml', OUTPUT_ROOT)

In [8]:
torch.cuda.is_available()

True

## make path list

- __train_paths__ ([dict]): Each Machine Type
    - __ToyCar__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) :paths
    - __ToyTrain__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) : paths
    - ...

In [9]:
dev_paths = {}

for machine_type in machine_types:
    # dev train
    dev_train_paths = [f"{dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/train")]
    dev_train_paths = sorted(dev_train_paths)
    
    print(f'Machine {machine_type}', end=',')
    print(f'dev_train_size : {len(dev_train_paths)}', end=', ')
    dev_train_source_paths = [path for path in dev_train_paths if 'source' in path]
    print(f'source_size : {len(dev_train_source_paths)}', end=', ')
    dev_train_target_paths = [path for path in dev_train_paths if 'target' in path]
    print(f'target_size : {len(dev_train_target_paths)}', end=', ')
    # source mode
    dev_train_paths = dev_train_source_paths + dev_train_target_paths
    
    # add_dev train
    add_dev_paths = [f"{add_dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{add_dev_dir}/{machine_type}/train")]
    add_dev_paths = sorted(add_dev_paths)
    print(f'adddev_train_size : {len(dev_train_paths)}', end=', ')
    add_dev_source_paths = [path for path in add_dev_paths if 'source' in path]
    print(f'source_size : {len(add_dev_source_paths)}', end=', ')
    add_dev_target_paths = [path for path in add_dev_paths if 'target' in path]
    print(f'target_size : {len(add_dev_target_paths)}')
    # source mode
    add_dev_paths = add_dev_source_paths + add_dev_target_paths
    print('==============================================================')
    
    # dev_source valid
    dev_source_paths = [f"{dev_dir}/{machine_type}/source_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/source_test")]
    dev_source_paths = sorted(dev_source_paths)
    
    # dev_target valid
    dev_target_paths = [f"{dev_dir}/{machine_type}/target_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/target_test")]
    dev_target_paths = sorted(dev_target_paths)
    
    # bundle
    dev_paths[machine_type] = {}
    dev_paths[machine_type]['train'] = dev_train_paths + add_dev_paths
    dev_paths[machine_type]['valid_source'] = dev_source_paths
    dev_paths[machine_type]['valid_target'] = dev_target_paths

Machine pump,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine gearbox,dev_train_size : 3026, source_size : 3017, target_size : 9, adddev_train_size : 3026, source_size : 3105, target_size : 9
Machine slider,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyCar,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine valve,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyTrain,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine fan,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9


In [10]:
machine_types

['pump', 'gearbox', 'slider', 'ToyCar', 'valve', 'ToyTrain', 'fan']

In [11]:
#############################################################################
# run
#############################################################################
def run(machine_type, dev_paths):
    com.tic()
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(dev_paths, machine_type)
    # parameter setting
    logger.info('TRAINING')
    model = Model(n_out=36, n_centers=6)
    
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
    output_dict, model, pred_df, score_df = modeler.run_training(model, dataloaders_dict, writer, optimizer)
    
    # output
    feature_out_path = f'{OUT_FEATURE_DIR}/{machine_type}_features.pkl'
    model_out_path = f'{MODEL_DIR}/{machine_type}_model.pkl'
    pred_out_path = f'{PRED_DIR}/{machine_type}_pred.csv'
    score_out_path = f'{SCORE_DIR}/{machine_type}_pred.csv'
    # save
    pd.to_pickle(output_dict, feature_out_path)
    torch.save(model.state_dict(), model_out_path)
    pred_df.to_csv(pred_out_path)
    score_df.to_csv(score_out_path)
    logger.info(f'SAVE SUCCESS : {model_out_path}')
    
    com.toc()

In [12]:
#run(machine_types[0], dev_paths)

In [13]:
for machine_type in machine_types[0:]:
    run(machine_type, dev_paths)

2021-10-16 08:00:46,814 - 00_train.py - INFO - TARGET MACHINE_TYPE: pump
2021-10-16 08:00:46,815 - 00_train.py - INFO - MAKE DATA_LOADER
2021-10-16 08:00:46,819 - 00_train.py - INFO - TRAINING


use: cuda:0


100%|██████████| 600/600 [01:05<00:00,  9.10it/s]
2021-10-16 08:03:28,373 - pytorch_modeler.py - INFO - epoch:1/100, tr_loss:54720.564055, src_loss:7.609589, src_mean_auc:0.459300, tgt_loss:7.622005, tgt_mean_auc:0.509767,


,AUC,pAUC
Source_0,0.499400,0.481579
Source_1,0.391600,0.492105
Source_2,0.486900,0.506316
Target_0,0.581800,0.507895
Target_1,0.460100,0.507895
Target_2,0.487400,0.498421
mean,0.484533,0.499035
h_mean,0.477943,0.498844


100%|██████████| 24/24 [00:26<00:00,  1.11s/it]
2021-10-16 08:03:55,019 - pytorch_modeler.py - INFO - epoch:2/100, tr_loss:163928.507638, src_loss:7.609589, src_mean_auc:0.459300, tgt_loss:7.622005, tgt_mean_auc:0.509767,
100%|██████████| 24/24 [00:26<00:00,  1.11s/it]
2021-10-16 08:04:21,699 - pytorch_modeler.py - INFO - epoch:3/100, tr_loss:186583.301749, src_loss:7.609589, src_mean_auc:0.459300, tgt_loss:7.622005, tgt_mean_auc:0.509767,
100%|██████████| 24/24 [00:26<00:00,  1.11s/it]
2021-10-16 08:04:48,376 - pytorch_modeler.py - INFO - epoch:4/100, tr_loss:1112504.706708, src_loss:7.609589, src_mean_auc:0.459300, tgt_loss:7.622005, tgt_mean_auc:0.509767,
100%|██████████| 24/24 [00:26<00:00,  1.11s/it]
2021-10-16 08:05:14,976 - pytorch_modeler.py - INFO - epoch:5/100, tr_loss:126966.391233, src_loss:7.609589, src_mean_auc:0.459300, tgt_loss:7.622005, tgt_mean_auc:0.509767,
100%|██████████| 24/24 [00:27<00:00,  1.16s/it]
2021-10-16 08:05:42,802 - pytorch_modeler.py - INFO - epoch:6/1

,AUC,pAUC
Source_0,0.486950,0.486316
Source_1,0.443650,0.484737
Source_2,0.452900,0.505789
Target_0,0.559100,0.508947
Target_1,0.425450,0.483684
Target_2,0.523200,0.521053
mean,0.481875,0.498421
h_mean,0.477480,0.498014


100%|██████████| 24/24 [00:26<00:00,  1.12s/it]
2021-10-16 08:10:35,670 - pytorch_modeler.py - INFO - epoch:12/100, tr_loss:969594.942515, src_loss:7.780938, src_mean_auc:0.461167, tgt_loss:7.643982, tgt_mean_auc:0.502583,
100%|██████████| 24/24 [00:26<00:00,  1.12s/it]
2021-10-16 08:11:02,569 - pytorch_modeler.py - INFO - epoch:13/100, tr_loss:111123.436467, src_loss:7.780938, src_mean_auc:0.461167, tgt_loss:7.643982, tgt_mean_auc:0.502583,
100%|██████████| 24/24 [00:27<00:00,  1.15s/it]
2021-10-16 08:11:30,266 - pytorch_modeler.py - INFO - epoch:14/100, tr_loss:511316.892522, src_loss:7.780938, src_mean_auc:0.461167, tgt_loss:7.643982, tgt_mean_auc:0.502583,
100%|██████████| 24/24 [00:26<00:00,  1.11s/it]
2021-10-16 08:11:56,993 - pytorch_modeler.py - INFO - epoch:15/100, tr_loss:52164.603289, src_loss:7.780938, src_mean_auc:0.461167, tgt_loss:7.643982, tgt_mean_auc:0.502583,
100%|██████████| 24/24 [00:26<00:00,  1.12s/it]
2021-10-16 08:12:23,834 - pytorch_modeler.py - INFO - epoch:1

,AUC,pAUC
Source_0,0.549400,0.495789
Source_1,0.552400,0.543684
Source_2,0.527300,0.547368
Target_0,0.555000,0.525263
Target_1,0.415000,0.488947
Target_2,0.489300,0.511053
mean,0.514733,0.518684
h_mean,0.509248,0.517733


100%|██████████| 24/24 [00:28<00:00,  1.17s/it]
2021-10-16 08:17:18,981 - pytorch_modeler.py - INFO - epoch:22/100, tr_loss:95739.656019, src_loss:46.180496, src_mean_auc:0.543033, tgt_loss:29.899119, tgt_mean_auc:0.486433,
100%|██████████| 24/24 [00:27<00:00,  1.14s/it]
2021-10-16 08:17:46,409 - pytorch_modeler.py - INFO - epoch:23/100, tr_loss:89048.734411, src_loss:46.180496, src_mean_auc:0.543033, tgt_loss:29.899119, tgt_mean_auc:0.486433,
100%|██████████| 24/24 [00:26<00:00,  1.11s/it]
2021-10-16 08:18:13,091 - pytorch_modeler.py - INFO - epoch:24/100, tr_loss:139664.795066, src_loss:46.180496, src_mean_auc:0.543033, tgt_loss:29.899119, tgt_mean_auc:0.486433,
100%|██████████| 24/24 [00:26<00:00,  1.10s/it]
2021-10-16 08:18:39,507 - pytorch_modeler.py - INFO - epoch:25/100, tr_loss:nan, src_loss:46.180496, src_mean_auc:0.543033, tgt_loss:29.899119, tgt_mean_auc:0.486433,
 38%|███▊      | 9/24 [00:10<00:17,  1.16s/it]


KeyboardInterrupt: 

In [ ]:
#run(machine_types[0], dev_paths)

# run